In [ ]:

!pip install keras==2.8.0

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
import numpy as np
import pandas as pd
#from tensorflow import keras
import cv2
from matplotlib import pyplot as plt
import os 
import random 
from PIL import Image

In [ ]:
df=pd.read_csv("train.csv")

In [ ]:
df

In [ ]:
samples=20000
df=df.loc[:samples,:]
num_classes = len(df["landmark_id"].unique())
num_data= len(df)

In [ ]:
num_classes


In [ ]:
num_data


In [ ]:
data=pd.DataFrame(df["landmark_id"].value_counts())

data.reset_index(inplace=True)
data.head()

In [ ]:
data.tail()

In [ ]:
data.columns=['index','landmark_id']

In [ ]:
data['landmark_id'].describe()

In [ ]:
plt.hist(data['landmark_id'],100,range=(0,58),label='test')

In [ ]:
data['landmark_id'].between(0,5).sum()
data['landmark_id'].between(5,10).sum()
data['landmark_id'].between(10,15).sum()
data['landmark_id'].between(15,20).sum()

In [ ]:
plt.hist(df["landmark_id"], bins=df["landmark_id"].unique())

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labelencoder.fit(df["landmark_id"])

In [ ]:
df


In [ ]:
filtered_df = df[df['id'].str.startswith('000')]
df = filtered_df

In [ ]:
def encode_label(label):
 return labelencoder.transform(label)

In [ ]:
def decode_label(label):
 return labelencoder.inverse_transform(label)

In [ ]:
import os
import cv2
def get_img_from_num(num, df):
   if 0 <= num < len(df): # Check if the index is within the DataFrame length
     fname, label = df.iloc[num, :]
     print(fname)
     f1, f2, f3 = fname[:3] # Extracting first three characters
     path = os.path.join("./images", f"images_{f1}{f2}{f3}", f1, f2, f3, f"{fname}")
    try:
      img = cv2.imread(path)
      if img is not None:
        print(img)
        return img, label
      else:
        print(f"Error: Unable to read image at path: {path}")
        return None, None
    except Exception as e:
    print(f"Error: {e}")
    return None, None
   else:
    print("Error: Index out of bounds or DataFrame length is insufficient.")
    return None, None

In [ ]:
fig = plt.figure(figsize=(16, 16))

for i in range(1, 5):
    rimg = random.choices(os.listdir("./images/images_000/0/0"), k=3)
    print(rimg)
    
    folder = "./images/images_000/0/0" + "/" + rimg[2]
    random_img = random.choice(os.listdir(folder))
    
    iimg = np.array(Image.open(folder + "/" + random_img))
    fig.add_subplot(1, 4, i)
    plt.imshow(iimg)
    plt.axis('off')

plt.show()

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.layers import *
from keras import Sequential

In [ ]:
#Parameters
learning_rate = 0.001
decay_speed = 1e-6
momentum = 0.09
loss_function = "sparse_categorical_crossentropy"
source_model = VGG19(weights=None)
drop_layer = Dropout(0.5)
drop_layer2 = Dropout(0.5)

In [ ]:
model = Sequential()
for layer in source_model.layers[:-1]:
   if layer == source_model.layers[-25]:
        model.add(BatchNormalization())
    model.add(layer)
model.add(Dense(num_classes, activation="softmax"))
model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.RMSprop(lr=learning_rate),
              loss=loss_function,
              metrics=['accuracy'])

In [ ]:
def image_reshape(img, size):
    return cv2.resize(img, size)

In [ ]:
def get_batch(dataframe, start, batch_size):
    image_array = []
    label_array = []
    
    end_image = start + batch_size
    if(end_image) > len(dataframe):
        rend_image = len(dataframe)

    for idx in range(start, end_image):
        n = idx
        img, label = get_img_from_num(n, dataframe)
        img = image_reshape(img, (224, 224)) / 255.0
        image_array.append(img)
        label_array.append(label)
    label_array = encode_label(label_array)

    return np.array(image_array), np.array(label_array)

In [ ]:
batch_size = 64
epoch_shuffle = True
weight_classes = True
epochs = 1
#Splitting the data with 80% of length
train, val = np.split(df.sample(frac=1),[int(0.8*len(df))])
print(len(train))
print(len(val))

In [ ]:

df

In [ ]:
import cv2
for e in range(epochs):
    print("Epoch : " + str(e+1) + "/" + str(epochs))
    if epoch_shuffle:
        train = train.sample(frac = 1)
    for it in range(int(np.ceil(len(train)/batch_size))):
        X_train, y_train = get_batch(train, it*batch_size, batch_size)
        model.train_on_batch(X_train, y_train)

In [ ]:
# Test
batch_size = 16
errors = 0
good_preds = []
bad_preds = []
for it in range(int(np.ceil(len(val)/batch_size))):
    X_val, y_val = get_batch(val, it*batch_size, batch_size)


    print("Validation Image paths:", X_val)

    result = model.predict(X_val)
    cla = np.argmax(result, axis=1)
    for idx, res in enumerate(result):
        if cla[idx] != y_val[idx]:
            errors = errors + 1
            bad_preds.append([batch_size*it + idx, cla[idx], res[cla[idx]]])
        else:
            good_preds.append([batch_size*it + idx, cla[idx], res[cla[idx]]])
model.save("Model")

In [ ]:
good_preds = np.array(good_preds)
good_preds = np.array(sorted(good_preds, key = lambda x: x[2], reverse=True))

In [ ]:
fig = plt.figure(figsize=(16, 16))
for i in range(1, 6):
    if i < len(good_preds):
        n = int(good_preds[i][0])
        img, lbl = get_image_from_number(n, val)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        fig.add_subplot(1, 5, i)
        plt.imshow(img)
        lbl2 = np.array(int(good_preds[i][1])).reshape(1, 1)
        sample_cnt = list(df.landmark_id).count(lbl)
        plt.title("Label: " + str(lbl) + "\nClassified as: " + str(decode_label(lbl
        plt.axis('off')
        plt.show()